In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [ ]:
df = pd.read_csv('Jan-April-20.csv')
df.head()

,state_residence,district_residence,Date,new_case,negative_case,Failed_test,Total_test,Invalid,Rejected,Rejected (sample leakage),-select result-
0,ANDAMAN AND NICOBAR ISLANDS,NICOBARS,2021-01-11,0,14,0,14,NaN,NaN,NaN,NaN
1,ANDAMAN AND NICOBAR ISLANDS,NICOBARS,2021-01-12,0,15,0,15,NaN,NaN,NaN,NaN
2,ANDAMAN AND NICOBAR ISLANDS,NICOBARS,2021-01-13,0,13,0,13,NaN,NaN,NaN,NaN
3,ANDAMAN AND NICOBAR ISLANDS,NICOBARS,2021-01-14,1,3,0,4,NaN,NaN,NaN,NaN
4,ANDAMAN AND NICOBAR ISLANDS,NICOBARS,2021-01-15,0,20,0,20,NaN,NaN,NaN,NaN


In [ ]:
STATES = []
for index, row in df.iterrows():
    if row["state_residence"] not in STATES:
      STATES.append(row["state_residence"])

state_wise_data_total = dict()
state_wise_data_positive = dict()
state_wise_data_date = dict()

for state in STATES:
  state_wise_data_total[state] = []
  state_wise_data_positive[state] = []
  state_wise_data_date[state] = []
  for index, row in df.iterrows():
    state_wise_data_total[state].append(row["Total_test"])
    state_wise_data_positive[state].append(row["Total_test"] - row["negative_case"] - row["Failed_test"])
    state_wise_data_date[state].append(row["Date"])

state_wise_data_positive_x_train = dict()
state_wise_data_positive_y_train = dict()
scaled_data = state_wise_data_positive
scaler = MinMaxScaler(feature_range=(0, 1))

prediction_days = 4

model = dict()

for state in STATES:
  state_wise_data_positive[state] = np.array(state_wise_data_positive[state])
  scaled_data[state] = scaler.fit_transform(state_wise_data_positive[state].reshape(-1, 1))
  x_train, y_train = [], []
  
  for x in range(prediction_days, len(scaled_data[state])):
    x_train.append(scaled_data[state][x - prediction_days: x, 0])
    y_train.append(scaled_data[state][x, 0])
  
  x_train, y_train = np.array(x_train), np.array(y_train)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

  state_wise_data_positive_x_train[state] = x_train
  state_wise_data_positive_y_train[state] = y_train

  model[state] = Sequential()
  model[state].add(LSTM(units = 50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
  model[state].add(Dropout(0.2))
  model[state].add(LSTM(units = 50, return_sequences=True))
  model[state].add(Dropout(0.2))

  model[state].add(LSTM(units = 50))
  model[state].add(Dropout(0.2))
  model[state].add(Dense(units = 1))  #our predictions
  model[state].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
index = 0
for state in STATES:
  index += 1
  # if (index >= 2):
  #   break
  print("Doing for state:",index, "=>", state)
  model[state].fit(x_train, y_train, epochs = 25, batch_size = 32)

Doing for state: 1 => ANDAMAN AND NICOBAR ISLANDS
Epoch 1/25
2458/2458 [==============================] - 27s 7ms/step - loss: 4.5783e-04
Epoch 2/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.4892e-04
Epoch 3/25
2458/2458 [==============================] - 19s 8ms/step - loss: 3.2658e-04
Epoch 4/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.3475e-04
Epoch 5/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.1808e-04
Epoch 6/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.2688e-04
Epoch 7/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.2573e-04
Epoch 8/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.1071e-04
Epoch 9/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.1053e-04
Epoch 10/25
2458/2458 [==============================] - 18s 7ms/step - loss: 3.1767e-04
Epoch 11/25
2458/2458 [==============================] - 18s 7ms/step

In [ ]:
!mkdir -p saved_model
for state in STATES:
  model[state].save('saved_model/model' + str(state))

In [ ]:
index = 0
predictions = dict()
for state in STATES:
  index += 1
  # if index >= 2:
  #   break
  results = []

  for i in range(10):
    x_test = []
    x_test.append(scaled_data[state][-prediction_days:])

    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    predicted_cases = model[state].predict(x_test)
    scaled_data[state] = np.insert(scaled_data[state], len(scaled_data[state]), predicted_cases[0])
    predicted_cases = scaler.inverse_transform(predicted_cases).reshape(-1, )
    results.append(int(predicted_cases))
  
  predictions[state] = results

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
def plot_for_state(state):
  fig = plt.figure()
  plt.plot(predictions[state])
  plt.title("Covid Cases")
  plt.xlabel('Day')
  plt.ylabel('Cases')
  return fig

final = gr.Interface(plot_for_state, inputs=[gr.Dropdown(STATES)], outputs=gr.Plot())

In [ ]:
final.launch()